In [1]:
#Lib
import pandas as pd
from joblib import load
#Clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
#Zips
import py7zr
#elbow point
%pip install kneed
from kneed import KneeLocator
from sklearn.preprocessing import MinMaxScaler
# plots 
%pip install plotly
import plotly.express as px
import matplotlib.pyplot as plt
#Save plots as .png
%pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Defining target column
target = 'TARGET'
target_value = 1.0
name = 'telecom_churn_me'

In [3]:
#Accessing transformed data
with py7zr.SevenZipFile(f'../../../../data/{name}/fully_transformed.csv', mode='r') as z:
    z.extractall(path='fully_transformed/')

FileNotFoundError: [Errno 2] No such file or directory: '../../../../data/telecom_churn_me/fully_transformed.7z'

In [ ]:
#Reading transformed data
df = pd.read_csv('./fully_transformed/fully_transformed.csv')
df

,AccountWeeks,DataPlan,CustServCalls,Churn
0,0.249489,-0.618303,-1.188040,0
1,0.500606,1.616843,1.852649,0
2,-1.156765,-0.618303,-0.427868,0
3,-0.905648,-0.618303,0.332305,0
4,1.228845,-0.618303,-0.427868,0
...,...,...,...,...
3286,1.354403,1.616843,-0.427868,0
3287,2.509541,-0.618303,-0.427868,0
3288,0.852170,-0.618303,-0.427868,0
3289,-0.604308,-0.618303,1.092477,0


In [ ]:
#Preparing dataset for clustering
y = df[target]
x = df.drop([target], axis=1)
x

,AccountWeeks,DataPlan,CustServCalls
0,0.249489,-0.618303,-1.188040
1,0.500606,1.616843,1.852649
2,-1.156765,-0.618303,-0.427868
3,-0.905648,-0.618303,0.332305
4,1.228845,-0.618303,-0.427868
...,...,...,...
3286,1.354403,1.616843,-0.427868
3287,2.509541,-0.618303,-0.427868
3288,0.852170,-0.618303,-0.427868
3289,-0.604308,-0.618303,1.092477


In [ ]:
#Getting number of k-means clusters
def elbow(dataset):
    scaler = MinMaxScaler()
    scaler.fit(dataset)
    X=scaler.transform(dataset)
    inertia = []
    max_clusters = dataset.shape[1] + 1 #number of variables
    cluster_number = list(range(1, max_clusters))
    for i in cluster_number:
        kmeans = KMeans(
            n_clusters=i, init="k-means++",
            n_init=10,
            tol=1e-04, random_state=42
        )
        kmeans.fit(X)
        inertia.append(kmeans.inertia_)
    #locating elbow point
    kneedle = KneeLocator(cluster_number, inertia, S=1.0, curve="convex", direction="decreasing")
    clusters_number = kneedle.knee
    if (cluster_number != int):
        kneedle = KneeLocator(cluster_number, inertia, S=0.0, curve="convex", direction="decreasing")
        clusters_number = kneedle.knee
    return clusters_number

In [ ]:
clusters_number = elbow(x)
clusters_number

/opt/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning:

No knee/elbow found



2

In [ ]:
#Clustering function
def clustering(dataframe, clusters_number):
    dataframe.reset_index
    #k-means algorithm
    kmeans = KMeans(
            n_clusters=clusters_number, #number of clusters for general dataset
            init="k-means++",
            n_init=10,
            tol=1e-04, 
            random_state=42
        )
    #fitting the algorithm
    kmeans.fit(dataframe)
    #labeling
    clusters=pd.DataFrame(dataframe)
    clusters['label']=kmeans.labels_
    return clusters

In [ ]:
#Transformed dataframe labeled 
df = pd.DataFrame(clustering(x, clusters_number))
#Main clustering polar graph
polar=df.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig0 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig0.show()
fig0.write_image(f'main_cluster_img.png')
#Main clustering pie plot to see clustering distribution
pie0=df.groupby('label').size().reset_index()
pie0.columns=['label','value']
pie0 = px.pie(pie0,values='value',names='label')
pie0.show()
pie0.write_image(f'main_cluster_distribution.png')


/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
df.to_csv(f'./main_cluster.csv') #creating filtered cvs 
#Main dataframe clustered
df

,AccountWeeks,DataPlan,CustServCalls,label
0,0.249489,-0.618303,-1.188040,0
1,0.500606,1.616843,1.852649,1
2,-1.156765,-0.618303,-0.427868,0
3,-0.905648,-0.618303,0.332305,0
4,1.228845,-0.618303,-0.427868,0
...,...,...,...,...
3286,1.354403,1.616843,-0.427868,1
3287,2.509541,-0.618303,-0.427868,0
3288,0.852170,-0.618303,-0.427868,0
3289,-0.604308,-0.618303,1.092477,0


In [ ]:
#Create a csv focusing in cluster churns
clust = 0 #iterator
df[target] = y
while clust < clusters_number:
    dataframe = pd.DataFrame(df.loc[df['label'] == clust]) #Filter by cluster
    dataframe = dataframe.loc[dataframe[target] == target_value] #Filter by positive targets
    dataframe = dataframe.drop(columns=[target,'label'])
    dataframe.to_csv(f'./cluster{clust}.csv') #creating filtered cvs 
    clust = clust + 1 #iterator
dataframe

,AccountWeeks,DataPlan,CustServCalls
75,0.801946,1.616843,0.332305
133,1.203733,1.616843,2.612822
144,0.500606,1.616843,1.092477
172,-0.403414,1.616843,1.852649
191,0.626165,1.616843,1.852649
...,...,...,...
3093,-1.633887,1.616843,-1.188040
3148,1.078175,1.616843,0.332305
3156,1.153510,1.616843,0.332305
3158,-0.001627,1.616843,-0.427868


In [ ]:
clust = 0 #iterator
while clust < clusters_number:
    sub_cluster = pd.read_csv(f'./cluster{clust}.csv') #read every subcluster .csv
    sub_cluster = sub_cluster.drop(columns=['Unnamed: 0']) #drop the added column
    sub_clusters_number = elbow(sub_cluster) #calculate elbow point for every sub-cluster
    clusters = pd.DataFrame(clustering(sub_cluster, sub_clusters_number)) #k-means clustering method
    #polar sub-clusters graph
    sub_polar=clusters.groupby("label").mean().reset_index() 
    sub_polar=pd.melt(sub_polar,id_vars=["label"])
    fig = px.line_polar(sub_polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
    fig.show() #print here
    fig.write_image(f'cluster{clust}img.png') #save as .png file
    #Pie plot to see sub-cluster's distribution
    pie=clusters.groupby('label').size().reset_index()
    pie.columns=['label','value']
    pie = px.pie(pie,values='value',names='label')
    pie.show() #print here
    pie.write_image(f'cluster{clust}distribution.png') #save as .png file
    clust = clust + 1
    
    

/opt/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning:

No knee/elbow found

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



/opt/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning:

No knee/elbow found

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

